# Capstone Project - The Battle of the Neighborhoods

## Denver Brewery Location Analysis

### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>

This projects goal is to make compairisons of current beer brewery locations in downtown Denver, Colorado, U.S.  The focus is on exiting brewerys neighboring buisnesses and location features specifically for compairison. This report is designed to help buisness entreprenures make a better decision about where to potentialy locate a new beer brewery in Denver.

There are many sucessful brewerys in Denver Colorado, this project will attempt to decern why other brewerys chose their location and buisness neighbors. This project will attempt to point out what neigboring factors all the brewerys share and what factors are unique. Brewerys can be sucessful located near other brewerys, so its important to get a better understanding.

Using both map visualization and clusteringing this project will allow a buisness entrprenures to visualize current brewery locations for compairison while allowing for a deeper look at what current brewerys are located near or next to in specific locations. This will allow for a better dunderstanding of location specifics for the location of a new beer brewery.

## Data <a name="data"></a>

This projects discovery process will include:
* Geo location mapping of existing brewerys in Denver
* Neigboring buisness within 500 meters of existing brewerys
* Other neigbors such parks and monuments

A map of Denver will be populated with markers indication the physical location of current brewerys.

The following data sources will be needed to extract/generate the required information:
* Brewery geo location data and names from the BrewPub dataset.
* Brewery buisness neighbors and their type and location in Denver will be obtained using **Foursquare API**
* Coordinate for Denver  will be obtained using  **geopy library**


*** Note please insert you own Foursquare credentials to run project!!!***

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##  Importing the BrewPub Data set

This csv file was dowloaded directly from kaggle.com from the following url.

* Dataset: Breweries and Brew Pubs in the USA
* Information: A list of over 7,000 breweries and brew pubs in the USA.
* Data provider: Datafiniti
* Updated 3 years ago (Version 2)
* URL: https://www.kaggle.com/datafiniti/breweries-brew-pubs-in-the-usa?select=7160_1.csv

About This Data
* This is a list of over 7,000 breweries and brewpubs in the USA provided by Datafiniti's Business Database. 
* The dataset includes the category, name, address, city, state, and more for each listing.
* Note that this is a sample of a large dataset. The full dataset is available through Datafiniti.

In [2]:
# reads in csv file directly from machine
df = pd.read_csv("7160_1.csv")

In [3]:
# display head of data frame
df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,407 Radam Ln,brewery,Austin,US,us/tx/austin/407radamln,NaN,NaN,(512) Brewing Co,5127072337,78745,TX,NaN
1,1135 N W Galveston Ave,Brewery and Bar,Bend,US,us/or/bend/1135nwgalvestonave,NaN,NaN,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",97703,OR,10barrel.com
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,"18174031832, 817 403-1832",76051,TX,http://www.10gallontours.com/
4,Po Box 135,"American Restaurants, Bar & Grill Restaurants,...",Villard,US,us/mn/villard/pobox135,NaN,NaN,10 Mile Tavern,3205542939,56385,MN,http://www.10miletavern.com


In [4]:
# display information about column counts, row counts, and  data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7375 entries, 0 to 7374
Data columns (total 12 columns):
address       7375 non-null object
categories    7375 non-null object
city          7375 non-null object
country       7375 non-null object
key           7375 non-null object
lat           5414 non-null float64
long          5416 non-null float64
name          7375 non-null object
phones        6159 non-null object
postalCode    7272 non-null object
province      7375 non-null object
websites      3548 non-null object
dtypes: float64(2), object(10)
memory usage: 691.5+ KB


### Data preprocessing and cleaning

In [5]:
# creates a new dataframe using only brewerys located in Denver
df_den = df[(df['city'].str.contains('Denver',regex=False)) & (df['categories'].str.contains('Brewery', regex=False))].reset_index(drop=True)
df_den.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,1505 Ogden St,"Brewery, Brewery North Capitol Hill",Denver,US,us/co/denver/1505ogdenst,NaN,NaN,Alpine Dog Brewing Company,"(720) 214-5170, 7202145170",80218,CO,http://alpinedogbrewery.com
1,320 N Broadway,Brewery,Denver,US,us/co/denver/320nbroadway,39.721387,-104.986952,Baere Brewing Co.,3037333354,80203,CO,http://www.baerebrewing.com
2,3120 Blake St,"Brewery, Bar",Denver,US,us/co/denver/3120blakest/-793899859,NaN,NaN,Beryl's Beer Co.,7204200826,80205,CO,http://www.berylsbeerco.com/#DpOE79
3,3719 Walnut St,"Brewery and Performing Arts Venue Five Points,...",Denver,US,us/co/denver/3719walnutst,39.769782,-104.972869,Black Shirt Brewing Co.,"(303) 993-2799, 3039932799",80205,CO,http://blackshirtbrewingco.com
4,490 Santa Fe Dr,"Brewery and Beer Garden, Brewery, Beer Garden,...",Denver,US,us/co/denver/490santafedr,NaN,NaN,Black Sky Brewery,"7207085816, (720) 708-5816",80204,CO,http://blackskybrewing.com


In [6]:
# count of unique brewery names in the data frame
df_den['name'].nunique()

54

In [7]:
# display dataframe shape
df_den.shape

(54, 12)

In [8]:
# display column names
df_den.columns

Index(['address', 'categories', 'city', 'country', 'key', 'lat', 'long',
       'name', 'phones', 'postalCode', 'province', 'websites'],
      dtype='object')

### Dropping rows and columns
* Drop Boulder Beer Tap House; row 6 located at the Denver international airport
* The removed brewery is considerd an outlyer in this analysis due to its special location at the Denver International Airport.
* Columns to keep are  'name', 'lat', 'long'

In [9]:
# drop row 6 Boulder Beer Tap House at D.I.A in a new dataframe
df_den1 = df_den.drop([6], axis = 0)

In [10]:
# drops columns
df_den1 = df_den1.drop(['address', 'categories', 'country', 'key', 'phones', 'province', 'websites'], axis = 1)

In [11]:
# move name column to front of dataframe
df_den2 = df_den1.set_index('name')
# display new dataframe
df_den2.head()

,city,lat,long,postalCode
name,,,,
Alpine Dog Brewing Company,Denver,NaN,NaN,80218
Baere Brewing Co.,Denver,39.721387,-104.986952,80203
Beryl's Beer Co.,Denver,NaN,NaN,80205
Black Shirt Brewing Co.,Denver,39.769782,-104.972869,80205
Black Sky Brewery,Denver,NaN,NaN,80204


In [12]:
# drop all Nan values and reset the index in a new dataframe
df_den3 = df_den2.dropna().reset_index()

In [13]:
# display the new dataframe
df_den3.head()

,name,city,lat,long,postalCode
0,Baere Brewing Co.,Denver,39.721387,-104.986952,80203
1,Black Shirt Brewing Co.,Denver,39.769782,-104.972869,80205
2,Blue Moon Brewery At The Sandlot,Denver,39.756108,-104.992462,80205
3,Breckenridge Brewery Ballpark Pub,Denver,39.756351,-104.991315,80205
4,Breckenridge Colorado Craft,Denver,39.756381,-104.991360,80205


### Renaming Columns

In [14]:
# rename columns for better readability and selection purposes
df_den3 = df_den3.rename(columns={'name': 'Name','lat': 'Latitude', 'long': 'Longitude'})

In [15]:
# display the shape of the new data frame
df_den3.shape
# display the modified data frame
df_den3.head()

,Name,city,Latitude,Longitude,postalCode
0,Baere Brewing Co.,Denver,39.721387,-104.986952,80203
1,Black Shirt Brewing Co.,Denver,39.769782,-104.972869,80205
2,Blue Moon Brewery At The Sandlot,Denver,39.756108,-104.992462,80205
3,Breckenridge Brewery Ballpark Pub,Denver,39.756351,-104.991315,80205
4,Breckenridge Colorado Craft,Denver,39.756381,-104.991360,80205


## Exploring and Mapping Brewerys in Denver

* Use **geopy library** to get the latitude and longitude values of Denver Colorado
* In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent d_explorer, as shown below

In [16]:
# call location latitude and logitude for mapping Denver
address = 'Denver, Colorado'

geolocator = Nominatim(user_agent="d_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Denver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denver are 39.7392364, -104.9848623.


### Create a map of Denver with Brewerys superimposed on top

In [17]:
# create map of Denver using latitude and longitude values
map_denver = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Name in zip(df_den3['Latitude'], df_den3['Longitude'], df_den3['Name']):
    label = '{}'.format(Name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_denver)  
    
map_denver

# Foursquare API Data Preprocessing and Cleaning

In [18]:
# @hidden_cell
CLIENT_ID = 'my_client_id' # Foursquare ID
CLIENT_SECRET = 'my_client_secret' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
print("Credentials Excepted!")

Credentials Excepted!


### Exploring the name of the first Brewery in the df_den3 dataframe.

In [19]:
# locates and displays the first brewery name in the Name column
df_den3.loc[0, 'Name']

'Baere Brewing Co.'

### Retreiving and print the coordinates for the above Brewery

In [20]:
brewery_latitude = df_den3.loc[0, 'Latitude'] # brewery latitude value
brewery_longitude = df_den3.loc[0, 'Longitude'] # brewery longitude value

brewery_name = df_den3.loc[0, 'Name'] # brewery name

print('Latitude and Longitude values of the {} are {}, {}.'.format(brewery_name, 
                                                               brewery_latitude, 
                                                               brewery_longitude))

Latitude and Longitude values of the Baere Brewing Co. are 39.721387, -104.986952.


### Retrieve the top 100 venues that are near Baere Brewing Co. within a radius of 500 meters

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    brewery_latitude, 
    brewery_longitude, 
    radius, 
    LIMIT)

### Creating the GET request URL

In [22]:
# make the GET request
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eeacbe469babe001b3ac3f4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Central Denver',
  'headerFullLocation': 'Central Denver, Denver',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 39.7258870045, 'lng': -104.98111238763086},
   'sw': {'lat': 39.7168869955, 'lng': -104.99279161236915}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b3e49f6f964a520bb9a25e3',
       'name': 'CorePower Yoga',
       'location': {'address': '277 Broadway',
        'crossStreet': 'at 3rd Ave',
        'lat': 39.7208055,
        'lng': -104.9877806,
        'la

### Using the the get_category_type function from the Foursquare lab

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the json file and structure it into a pandas dataframe

In [24]:
venues = results['response']['groups'][0]['items']
    
# flatten JSON file
nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,CorePower Yoga,Yoga Studio,39.720805,-104.987781
1,Baere Brewing Co.,Brewery,39.721379,-104.987069
2,Illuzion Glass,Smoke Shop,39.720131,-104.987351
3,Adorn Salon Denver,Salon / Barbershop,39.720191,-104.987518
4,Studio Lites,Lingerie Store,39.721108,-104.988232


### Count of venues returned by Foursquare.

In [25]:
# print venue count
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


### A function to repeat the same venue function process to all the Brewerys in Denver

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Brewery Latitude', 
                  'Brewery Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Code to run the above function on each Brewery and create a new dataframe called denver_venues

In [27]:
# run getNearbyVenues function on all brewerys in dataset and create new dataframe
denver_venues = getNearbyVenues(names=df_den3['Name'],
                                   latitudes=df_den3['Latitude'],
                                   longitudes=df_den3['Longitude']
                                  )

Baere Brewing Co.
Black Shirt Brewing Co.
Blue Moon Brewery At The Sandlot
Breckenridge Brewery Ballpark Pub
Breckenridge Colorado Craft
Bull & Bush Pub and Brewery
Call To Arms Brewing Company
Crooked Stave @ The Source
Crooked Stave Barrel Cellar
Declaration Brewing
Denver Beer Co., Denver Beer Company
Denver Chophouse & Brewery
Diebolt Brewing Co.
Epic Brewing Denver
Ferm��ntra
Fiction Beer Company
Goldspot Brewing Co, Goldspot Brewing Company
Grandma's House
Great Divide Barrel Bar
Great Divide Brewery
Heavenly Daze Brewery (closed)
Hogshead Brewery
Jagged Mountain Brewery
Lost Highway Brewing Co
Mockery Brewing
Pints Pub
Prost Brewing
Ratio Beerworks
Renegade Brewing Co.
River North Brewery
Station 26 Brewing Company
Stem Ciders
Strange Craft Beer Company
Trve Brewing Co.
Vine Street Pub & Brewery
Wynkoop Brewing Co.
Zephyr Brewing


In [28]:
# display new dataframe shape
print(denver_venues.shape)
# dispalay new dataframe head
denver_venues.head()

(1417, 7)


,Name,Brewery Latitude,Brewery Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Baere Brewing Co.,39.721387,-104.986952,CorePower Yoga,39.720805,-104.987781,Yoga Studio
1,Baere Brewing Co.,39.721387,-104.986952,Baere Brewing Co.,39.721379,-104.987069,Brewery
2,Baere Brewing Co.,39.721387,-104.986952,Illuzion Glass,39.720131,-104.987351,Smoke Shop
3,Baere Brewing Co.,39.721387,-104.986952,Adorn Salon Denver,39.720191,-104.987518,Salon / Barbershop
4,Baere Brewing Co.,39.721387,-104.986952,Studio Lites,39.721108,-104.988232,Lingerie Store


### Count of venues returned for each Brewery by Name

In [29]:
# display count of venues based on name
denver_venues.groupby('Name').count()

,Brewery Latitude,Brewery Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Name,,,,,,
Baere Brewing Co.,48,48,48,48,48,48
Black Shirt Brewing Co.,29,29,29,29,29,29
Blue Moon Brewery At The Sandlot,40,40,40,40,40,40
Breckenridge Brewery Ballpark Pub,38,38,38,38,38,38
Breckenridge Colorado Craft,38,38,38,38,38,38
Bull & Bush Pub and Brewery,7,7,7,7,7,7
Call To Arms Brewing Company,37,37,37,37,37,37
Crooked Stave @ The Source,28,28,28,28,28,28
Crooked Stave Barrel Cellar,10,10,10,10,10,10


### Unique categories curated from all the returned venues

In [30]:
# print count of unique categorys 
print('There are {} uniques categories.'.format(len(denver_venues['Venue Category'].unique())))

There are 209 uniques categories.


## Methodology <a name="methodology"></a>

This projects effort will be directed at discovering information about existing breweries in Denver. This project will attempt to discern what makes a breweries location desirable and/or unique as well as considering a new breweries ability to be successful. This analysis will be limited to only breweries located within the city limits of Denver Colorado, U.S. 

The first step of this project was the collection of the required data. This data includes brewery geolocation data as well as names and addresses. The Brewpub data set corresponds with identified Denver breweries according to the Foursquare categorization of brewery.

The second step of the analysis is a visualization of breweries from the data superimposed on top of a map of Denver. This step provides visual cues such as distancing a brewery from one another in specific locations such as downtown Denver or randomly around the city. It also provides a way to see what neighborhoods the breweries are located. It also shows what intersections or primary streets the breweries are located on in order to generate foot traffic or curbside appeal.

The third step of this analysis is determining the type of business neighbors located within a 500 m radius of each operating brewery. Next is the count of these unique business types that are located near each brewery. And finally, is discovering similarities among breweries; and how these similarities can create clusters. This final step involves clustering breweries on a map of Denver using the K means clustering algorithm to determine similarities among the venues returned from the foursquare API. 

The fourth and final step of this analysis counting and looking at the top five most common venues for each brewery. This is then followed by clustering the top 10 venues for each brewery. These clusters are then overlaid on top of the map of Denver so that they are visually represented. Each cluster is then examined and discriminating venue category that distinguishes each cluster is provided based on the defining category. Each defining category is given a name that indicates its first most common venue in the cluster.

## Analysis <a name="analysis"></a>

One hot encoding is a way to take a count of existing venues as well as there absence. These counts will be used to find the top 5 and top 10 venues located near the brewerys in the data set. These counts will be used to create a new dataframe that will be combined with the brewery dataset for analysis.

### One-hot encoding of venue categories

In [31]:
# one hot encoding
denver_onehot = pd.get_dummies(denver_venues[['Venue Category']], prefix="", prefix_sep="")

# add brewery column back to dataframe
denver_onehot['Name'] = denver_venues['Name'] 

# move brewery column to the first column
cols = denver_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Name')))
denver_onehot = denver_onehot.reindex(columns = cols)

denver_onehot.head()

,Name,Accessories Store,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Movie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Baere Brewing Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Baere Brewing Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Baere Brewing Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
# display shape of new dataframe
denver_onehot.shape

(1417, 210)

### Group rows by Brewery and by taking the mean of the frequency of occurrence of each category

In [33]:
# group brewerys by mean of frequency and reset index
denver_grouped = denver_onehot.groupby('Name').mean().reset_index()
denver_grouped

,Name,Accessories Store,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Movie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Park,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Baere Brewing Co.,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.020833,0.000000,0.041667,0.000000,0.000000,0.000000,0.020833,0.000000,0.020833,0.020833,0.000000,0.020833,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020833,0.000000,0.041667,0.000000,0.000000,0.000000,0.0,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.020833,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.020833,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.020833,0.020833,0.000000,0.

In [34]:
# display shape of new dataframe
denver_grouped.shape

(37, 210)

### Print each Brewery along with the top 5 most common venue

In [35]:
# loop that prints out the Denver brewerys with its top 5 venues
num_top_venues = 5

for hood in denver_grouped['Name']:
    print("----"+hood+"----")
    temp = denver_grouped[denver_grouped['Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baere Brewing Co.----
                 venue  freq
0              Brewery  0.06
1   Mexican Restaurant  0.06
2                  Bar  0.06
3  American Restaurant  0.04
4          Coffee Shop  0.04


----Black Shirt Brewing Co.----
               venue  freq
0        Art Gallery  0.07
1        Music Venue  0.03
2  Convenience Store  0.03
3       Gourmet Shop  0.03
4         Food Court  0.03


----Blue Moon Brewery At The Sandlot----
            venue  freq
0     Yoga Studio  0.05
1  Sandwich Place  0.05
2         Brewery  0.05
3             Bar  0.05
4   Hot Dog Joint  0.05


----Breckenridge Brewery Ballpark Pub----
            venue  freq
0     Pizza Place  0.08
1             Bar  0.08
2         Brewery  0.05
3  Sandwich Place  0.05
4   Hot Dog Joint  0.05


----Breckenridge Colorado Craft----
            venue  freq
0     Pizza Place  0.08
1             Bar  0.08
2         Brewery  0.05
3  Sandwich Place  0.05
4   Hot Dog Joint  0.05


----Bull & Bush Pub and Brewery----
         

#### The above data is put into a pandas dataframe below.

### A function is defined to return the most common venues returned sorted

In [36]:
# most common venues function defined
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### The new dataframe created with the top 10 venues for each Brewery

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brewery_venues_sorted = pd.DataFrame(columns=columns)
brewery_venues_sorted['Name'] = denver_grouped['Name']

for ind in np.arange(denver_grouped.shape[0]):
    brewery_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denver_grouped.iloc[ind, :], num_top_venues)

brewery_venues_sorted.head()

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baere Brewing Co.,Brewery,Bar,Mexican Restaurant,Coffee Shop,American Restaurant,Salon / Barbershop,Sandwich Place,Arts & Crafts Store,Yoga Studio,Juice Bar
1,Black Shirt Brewing Co.,Art Gallery,Coffee Shop,Burger Joint,Light Rail Station,Speakeasy,Lounge,Marijuana Dispensary,Brewery,Seafood Restaurant,Sake Bar
2,Blue Moon Brewery At The Sandlot,Yoga Studio,American Restaurant,Bar,Pizza Place,Lounge,Brewery,Hot Dog Joint,Sandwich Place,Grocery Store,Pub
3,Breckenridge Brewery Ballpark Pub,Pizza Place,Bar,Sandwich Place,Hot Dog Joint,Lounge,Brewery,Sports Bar,Beer Garden,Bike Rental / Bike Share,Bistro
4,Breckenridge Colorado Craft,Pizza Place,Bar,Sandwich Place,Hot Dog Joint,Lounge,Brewery,Sports Bar,Beer Garden,Bike Rental / Bike Share,Bistro


### Kmeans for Clustering Brewerys

* k=5 was determined by visualy looking at the map of Denver with the overlayed brewerys on line 18 and manualy visualizing apperent clusters as best practice method rather than an elbow method, as the elbow method can not determine a best k.

In [38]:
# set number of clusters
kclusters = 5

denver_grouped_clustering = denver_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(denver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 1, 1, 1, 2, 1, 0, 4, 3])

### Merging dataframes: Clusters, Top 10 venues for each Brewery

In [39]:
# add clustering labels
brewery_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denver_merged = df_den3

# merge denver_grouped with df_den3 to add latitude/longitude for each brewery
denver_merged = denver_merged.join(brewery_venues_sorted.set_index('Name'), on='Name')

# displays head of new data frame
denver_merged.head()

,Name,city,Latitude,Longitude,postalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baere Brewing Co.,Denver,39.721387,-104.986952,80203,3,Brewery,Bar,Mexican Restaurant,Coffee Shop,American Restaurant,Salon / Barbershop,Sandwich Place,Arts & Crafts Store,Yoga Studio,Juice Bar
1,Black Shirt Brewing Co.,Denver,39.769782,-104.972869,80205,3,Art Gallery,Coffee Shop,Burger Joint,Light Rail Station,Speakeasy,Lounge,Marijuana Dispensary,Brewery,Seafood Restaurant,Sake Bar
2,Blue Moon Brewery At The Sandlot,Denver,39.756108,-104.992462,80205,1,Yoga Studio,American Restaurant,Bar,Pizza Place,Lounge,Brewery,Hot Dog Joint,Sandwich Place,Grocery Store,Pub
3,Breckenridge Brewery Ballpark Pub,Denver,39.756351,-104.991315,80205,1,Pizza Place,Bar,Sandwich Place,Hot Dog Joint,Lounge,Brewery,Sports Bar,Beer Garden,Bike Rental / Bike Share,Bistro
4,Breckenridge Colorado Craft,Denver,39.756381,-104.991360,80205,1,Pizza Place,Bar,Sandwich Place,Hot Dog Joint,Lounge,Brewery,Sports Bar,Beer Garden,Bike Rental / Bike Share,Bistro


### Visualizing the resulting clusters superimposed on the map of Denver

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, name, cluster in zip(denver_merged['Latitude'], denver_merged['Longitude'], denver_merged['Name'],
                                            denver_merged['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
    
folium.TileLayer('stamentoner').add_to(map_clusters)#changes map to black and white to see clusters better
       
map_clusters

### Examine Clusters

* By examining each cluster and determine the discriminating venue categories that distinguish each cluster.
* Based on the defining categories, a name was created for each cluster.

### Cluster 0: Brewery

In [41]:
denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Crooked Stave @ The Source,0,Brewery,Art Gallery,Pizza Place,BBQ Joint,Building,Martial Arts Dojo,Sake Bar,Roof Deck,Lounge,Food & Drink Shop
13,Epic Brewing Denver,0,Brewery,Bar,Coffee Shop,Seafood Restaurant,Burger Joint,Rock Club,Restaurant,Café,Clothing Store,Cocktail Bar
18,Great Divide Barrel Bar,0,Brewery,Cocktail Bar,New American Restaurant,Art Gallery,Dive Bar,Restaurant,Food Court,Food & Drink Shop,Speakeasy,Dog Run
24,Mockery Brewing,0,Brewery,Cocktail Bar,Art Gallery,New American Restaurant,Tech Startup,Building,Liquor Store,Food & Drink Shop,Food Court,Roof Deck
27,Ratio Beerworks,0,Brewery,Bar,Burger Joint,Food Truck,Coffee Shop,Yoga Studio,Noodle House,Rock Club,Restaurant,Café
31,Stem Ciders,0,Brewery,Bar,Coffee Shop,Yoga Studio,Burger Joint,Food Truck,Café,Clothing Store,Noodle House,Seafood Restaurant
32,Strange Craft Beer Company,0,Brewery,Kitchen Supply Store,Herbs & Spices Store,Art Gallery,Storage Facility,Gym / Fitness Center,Event Space,Distillery,Fast Food Restaurant,Farmers Market
36,Zephyr Brewing,0,Brewery,Bar,Coffee Shop,Food Truck,Rock Club,Café,Cocktail Bar,Sushi Restaurant,Clothing Store,Italian Restaurant


### Cluster 1: Pizza Place

In [42]:
denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Blue Moon Brewery At The Sandlot,1,Yoga Studio,American Restaurant,Bar,Pizza Place,Lounge,Brewery,Hot Dog Joint,Sandwich Place,Grocery Store,Pub
3,Breckenridge Brewery Ballpark Pub,1,Pizza Place,Bar,Sandwich Place,Hot Dog Joint,Lounge,Brewery,Sports Bar,Beer Garden,Bike Rental / Bike Share,Bistro
4,Breckenridge Colorado Craft,1,Pizza Place,Bar,Sandwich Place,Hot Dog Joint,Lounge,Brewery,Sports Bar,Beer Garden,Bike Rental / Bike Share,Bistro
6,Call To Arms Brewing Company,1,Art Gallery,Brewery,Gift Shop,Outdoor Supply Store,Italian Restaurant,Breakfast Spot,Shopping Plaza,Farmers Market,Mexican Restaurant,Bakery
16,"Goldspot Brewing Co, Goldspot Brewing Company",1,Food & Drink Shop,Bar,Café,Music Venue,Sandwich Place,Brewery,Food Court,Marijuana Dispensary,Pizza Place,Intersection
19,Great Divide Brewery,1,Brewery,Pizza Place,Yoga Studio,Hot Dog Joint,Lounge,Sandwich Place,Sushi Restaurant,Mexican Restaurant,Gym,Bar
22,Jagged Mountain Brewery,1,Hotel,Bar,Pub,Brewery,Cocktail Bar,Coffee Shop,Hot Dog Joint,Lounge,Pizza Place,Yoga Studio
29,River North Brewery,1,Bar,Sushi Restaurant,Pizza Place,Hot Dog Joint,Burger Joint,Brewery,Coffee Shop,Café,Sports Bar,Karaoke Bar


### Cluster 2: Gym

In [43]:
denver_merged.loc[denver_merged['Cluster Labels'] == 2, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Bull & Bush Pub and Brewery,2,Gym,Park,Pub,Gym / Fitness Center,Yoga Studio,Discount Store,Farmers Market,Exhibit,Event Space,Ethiopian Restaurant


### Cluster 3: Food

In [44]:
denver_merged.loc[denver_merged['Cluster Labels'] == 3, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baere Brewing Co.,3,Brewery,Bar,Mexican Restaurant,Coffee Shop,American Restaurant,Salon / Barbershop,Sandwich Place,Arts & Crafts Store,Yoga Studio,Juice Bar
1,Black Shirt Brewing Co.,3,Art Gallery,Coffee Shop,Burger Joint,Light Rail Station,Speakeasy,Lounge,Marijuana Dispensary,Brewery,Seafood Restaurant,Sake Bar
9,Declaration Brewing,3,Fast Food Restaurant,Breakfast Spot,Yoga Studio,BBQ Joint,Brewery,Marijuana Dispensary,Mexican Restaurant,Food Truck,Rock Club,Light Rail Station
10,"Denver Beer Co., Denver Beer Company",3,Park,Café,Sushi Restaurant,Brewery,Coffee Shop,American Restaurant,New American Restaurant,Beer Garden,Wine Shop,Sporting Goods Shop
11,Denver Chophouse & Brewery,3,American Restaurant,Hotel,Cocktail Bar,Restaurant,Pizza Place,Bar,Italian Restaurant,Sandwich Place,Mexican Restaurant,Coffee Shop
12,Diebolt Brewing Co.,3,Mexican Restaurant,Brewery,Chinese Restaurant,Convenience Store,Liquor Store,Bar,Theater,Asian Restaurant,Pizza Place,Cosmetics Shop
14,Ferm��ntra,3,Coffee Shop,Breakfast Spot,Pizza Place,Café,Library,Stadium,Food Court,Soccer Field,Bookstore,Sandwich Place
15,Fiction Beer Company,3,Cosmetics Shop,Coffee Shop,Intersection,Sandwich Place,Hardware Store,Brewery,Chinese Restaurant,Recreation Center,Athletics & Sports,Yoga Studio
17,Grandma's House,3,Antique Shop,Marijuana Dispensary,Brewery,Thrift / Vintage Store,Yoga Studio,Creperie,Sandwich Place,Rental Car Location,Coffee Shop,Convenience Store
20,Heavenly Daze Brewery (closed),3,Convenience Store,Light Rail Station,Design Studio,Distillery,Breakfast Spot,Flea Market,Marijuana Dispensary,Bar,Hobby Shop,Hardware Store


### Cluster 4: Store

In [45]:
denver_merged.loc[denver_merged['Cluster Labels'] == 4, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Crooked Stave Barrel Cellar,4,Grocery Store,Convenience Store,Winery,Cosmetics Shop,Discount Store,Sandwich Place,Brewery,Construction & Landscaping,Gymnastics Gym,Farmers Market


## Results and Discussion <a name="results"></a>

This project's analysis shows that there are similarities among Denver brewery locations. There are both clusters of breweries near downtown Denver as well as independent breweries located around the city. The highest concentration of breweries was in downtown Denver. These breweries were visually only 1 block away on average from one another and are located on the same or parallel streets in the lower downtown or the Lodo borough. 
Other similarities found are that breweries are often located near Bars. The highest concentration of breweries near bars is in downtown Denver. Other factors that seem to have a high concentration of brewers around it are food or restaurants. This analysis used k=5 as its centroid start for clustering. The following clusters have been named by the corresponding concentration of vendors. Each cluster in the discriminating venue category will be described in the following section.

* Cluster 0: Breweries
The first most common venue or discriminating venue is other Breweries. In this cluster, the second most common venue is bars followed by food in the third most common venue position.


* Cluster 1: Pizza Place
The first most common venue or discriminating venue is Pizza place. In this cluster, the second most common venue is bars followed by food in the third most common venue position. The second and third most common venues are like cluster 0.


* Cluster2: Gym
The first most common venue for cluster two is a gym followed by a park in the second most common venue position. There’s only one brewery listed in this cluster. The name of the brewery is bull in Bush problem brewery which is in a specially built business Pavilion. This cluster appears to take care of an outlier in the data set that may not be representative of brewery location selection.


* Cluster3: Food
In cluster number three the first most common venue has returned several types of food locations. These food location venues include American restaurants, bagel shop, pizza place, sandwich place, food truck. The second most common venues are more food locations mixed in with breweries and bars. 


* Cluster 4: Grocery store
The first most common venue in the cluster is a grocery store followed by a convenience store in the second most common venue position. Like cluster 2 this clustering appears to take care of an outlier data point. There is only one brewery listed which is the Crooked Stave Barrel Cellar. The third most common venue for this location is a winery. Because this data point may be considered an outlier it may not be representative for brewery location selection.

When looking at the results of cluster zero resulting in other breweries it was important to look at the clusters superimposed on the map of Denver. When visually looking at cluster zero it can be concluded that these breweries are all in lower downtown or the LODO area which is known for its nightlife in Denver. Cluster number one seems to indicate that breweries are often located near food letters in downtown Denver. Cluster 1 also indicates that bars often business neighbors when breweries are in downtown Denver.

When it came to brewers that were not located in downtown Denver their business neighbors didn’t seem as important. There were a few cases where these breweries were located near food or bars, but most of the breweries located throughout the city seem to depend more upon busy streets or types of neighborhoods. This indicates that there is more research to be done when it comes to making location decisions about breweries that are not located in downtown Denver.

## Conclusion <a name="conclusion"></a>

This project was created to generate a better understanding of brewery locations in the city of Denver. This project can potentially help stakeholders identify potential new brewery locations. The goal of this project is to help stakeholders narrow down these potential brewery locations in a methodical fashion in order to identify these optimal locations for new brewery’s and information about exiting breweries.

This project does not recommend specific locations for potential breweries; instead this project helps stakeholders understand why existing breweries are built in their current locations while identifying key metrics and specific vendor information about business neighbors and their density. This project is a way for stakeholders to learn more about existing brewery location specifics such as cross streets, physical distancing between breweries, and other desirable location features that may have a bearing upon the location decision-making process.

Clustering of the location data for existing breweries combined with top 10 information on vendors was performed in order to create an informative map of brewery clusters alongside top 10 vendor counts that are used as key metrics in determining potential neighborhoods and locations to build a new brewery. The information and data generated by this project can be considered a starting point for location exploration for brewery startups and stakeholders.

Stakeholders can then decide on specific characteristics of specific neighborhoods and zones while adding information such as curbside appeal or proximity to specific city destinations. This projects data can be combined with other valuable information such as real estate availability, proximity to major streets and finally the economic and commercial viability of available locations.